In [11]:
import os
import numpy as np
import pydicom
import nibabel as nib
from tqdm import tqdm

In [25]:
# PATHS
preprocessed_base = "C:/Users/LENOVO/Downloads/CHAOS_Train_Sets/Train_Sets/preprocessed_data/min_max_normalized_images"
dataset_ready = "C:/Users/LENOVO/Downloads/CHAOS_Train_Sets/Train_Sets/dataset_ready"
os.makedirs(dataset_ready, exist_ok=True)

In [26]:
modalities = [
    "T2SPIR",
    "T1DUAL_InPhase",
    "T1DUAL_OutPhase"
]

# Create folders
for mod in modalities + ["multi_modal"]:
    os.makedirs(os.path.join(dataset_ready, mod,'images'), exist_ok=True)

In [27]:
# save nifti function
def save_nifti(volume, path):
    affine = np.eye(4) 
    nib.save(nib.Nifti1Image(volume, affine), path)

In [28]:
# Lister tous les patients (dans T2SPIR)
patients = sorted([p for p in os.listdir(os.path.join(preprocessed_base, "T2SPIR"))
                   if os.path.isdir(os.path.join(preprocessed_base, "T2SPIR", p))])

for patient in tqdm(patients, desc="Converting DICOM to NIfTI"):

    for modality in modalities:

        patient_mod_path = os.path.join(preprocessed_base, modality, patient)
        if not os.path.isdir(patient_mod_path):
            continue

        dicom_files = sorted([f for f in os.listdir(patient_mod_path) if f.lower().endswith(".dcm")])
        if not dicom_files:
            continue

        slices = []
        for f in dicom_files:
            ds = pydicom.dcmread(os.path.join(patient_mod_path, f))
            slices.append(ds.pixel_array)

        # Empiler slices → volume
        volume = np.stack(slices, axis=0).astype(np.float32)

        # Sauvegarde NIfTI dans la modalité
        out_path = os.path.join(dataset_ready, modality,"images", f"{patient}.nii.gz")
        save_nifti(volume, out_path)

        # Copier aussi dans le dossier multi-modal avec nom unique
        multi_out_path = os.path.join(dataset_ready, "multi_modal","images", f"{patient}_{modality}.nii.gz")
        save_nifti(volume, multi_out_path)

print("✅ Dataset ready, multi_modal folder contains all individual modality files.")



Converting DICOM to NIfTI:   0%|          | 0/20 [00:00<?, ?it/s]c:\Users\LENOVO\AppData\Local\Programs\Python\Python313\Lib\site-packages\pydicom\pixels\decoders\base.py:801: UserWarning: The number of bytes of pixel data is sufficient to contain 2 frames which is larger than the given (0028,0008) 'Number of Frames' value of 1. The returned data will include these extra frames and if it's correct then you should update 'Number of Frames' accordingly, otherwise pass 'allow_excess_frames=False' to return only the first 1 frames.
  warn_and_log(
Converting DICOM to NIfTI: 100%|██████████| 20/20 [01:33<00:00,  4.68s/it]

✅ Dataset ready, multi_modal folder contains all individual modality files.


In [ ]:
# Config
special_patients = ['13', '19', '2', '20', '3', '38']
resized_masks_path = r"C:/Users/LENOVO/Downloads/CHAOS_Train_Sets/Train_Sets/resized_data"
converted_masks_path = r"C:/Users/LENOVO/Downloads/CHAOS_Train_Sets/Train_Sets/converted_masks"
dataset_ready = "C:/Users/LENOVO/Downloads/CHAOS_Train_Sets/Train_Sets/dataset_ready"

modalities = ["T2SPIR", "T1DUAL_InPhase", "T1DUAL_OutPhase"]

# Process each modality separately
for modality in modalities:
    modality_dir = os.path.join(dataset_ready, modality)
    masks_dir = os.path.join(modality_dir, "masks")
    os.makedirs(masks_dir, exist_ok=True)

    # Determine base modality for masks (T1DUAL special case)
    base_mod = "T1DUAL" if "T1DUAL" in modality else modality

    # Get patients from preprocessed images
    images_dir = os.path.join(modality_dir, "images")
    patients = sorted([f.replace(".nii.gz","") for f in os.listdir(images_dir) if f.endswith(".nii.gz")])


    for patient in tqdm(patients, desc=f"Converting masks for {modality}"):

        # Determine mask folder
        # Determine mask folder
        if patient in special_patients:
            # Special patients → resized_masks
            if base_mod == "T1DUAL":
                mask_path = os.path.join(resized_masks_path, patient, base_mod, "Ground")
            else:
                mask_path = os.path.join(resized_masks_path, patient, base_mod)
        else:
            # Other patients → converted_masks
            mask_path = os.path.join(converted_masks_path, patient, base_mod)

        # Now mask_path is always defined
        if not os.path.isdir(mask_path):
            print(f"Warning: mask folder not found: {mask_path}")
            continue


        mask_files = sorted([f for f in os.listdir(mask_path) if f.lower().endswith((".png"))])
        if not mask_files:
            print(f"No mask files for patient {patient} in {mask_path}")
            continue

        slices = []
        for f in mask_files:
            img = Image.open(os.path.join(mask_path, f)).convert("L")
            slices.append(np.array(img))

        volume = np.stack(slices, axis=0).astype(np.float32)

        # Save mask
        out_path = os.path.join(masks_dir, f"{patient}_mask.nii.gz")
        save_nifti(volume, out_path)

# Optional: copy all images into a multi-modal folder without stacking
multi_modal_dir = os.path.join(dataset_ready, "multi_modal")
os.makedirs(multi_modal_dir, exist_ok=True)

for modality in modalities:
    modality_dir = os.path.join(dataset_ready, modality)
    for f in os.listdir(modality_dir):
        if f.endswith(".nii.gz"):
            src = os.path.join(modality_dir, f)
            dst = os.path.join(multi_modal_dir, f)
            if not os.path.exists(dst):
                os.rename(src, dst)  # or shutil.copy(src, dst) to keep original


Converting masks for T2SPIR: 0it [00:00, ?it/s]
Converting masks for T1DUAL_InPhase: 0it [00:00, ?it/s]
Converting masks for T1DUAL_OutPhase: 0it [00:00, ?it/s]
